In [10]:
from scipy.integrate import quad
from numpy import exp, sqrt, log
from math import pi
import cmath

def heston_call_price(kappa, theta, sigma, rho, v0 ,r ,T ,s0 ,K): 
    p1 = p(kappa, theta, sigma, rho, v0 ,r ,T ,s0 ,K, 1) 
    p2 = p(kappa, theta, sigma, rho, v0 ,r ,T ,s0 ,K, 2) 
    return (s0*p1-K*exp(-r*T)*p2) 

def p(kappa, theta, sigma, rho, v0 ,r ,T ,s0 , K, status): 
    integrand = lambda phi: (exp(-1j*phi*log(K))*f(phi, kappa, theta, sigma, rho, v0 , r, T, s0, status)/(1j*phi)).real 
    return (0.5+(1/pi)*quad(integrand,0,100)[0]) 

def f(phi, kappa, theta, sigma, rho, v0, r, T, s0, status): 
    if status==1: 
        u = 0.5 
        b = kappa-rho*sigma 
    else: 
        u = -0.5 
        b = kappa 
    a = kappa*theta 
    x = log(s0) 
    d = sqrt((rho*sigma*phi*1j-b)**2-sigma**2*(2*u*phi*1j-phi**2))  
    g = (b-rho*sigma*phi*1j+d)/(b-rho*sigma*phi*1j-d) 
    C = r*phi*1j*T+(a/sigma**2)*((b-rho*sigma*phi*1j+d)*T-2*log((1-g*exp(d*T))/( 1-g))) 
    D = (b-rho*sigma*phi*1j+d)/sigma**2*((1-exp(d*T))/(1-g*exp(d*T)))  
    return exp(C+D*v0+1j*phi*x)

In [19]:
T = 1
rho = -0.4
theta = 0.06
sigma = 0.1
kappa=3
v0 = 0.07
s0=115 
K=100
r = 0.01

t1 = time.time()
for i in range(10):
    print (heston_call_price(kappa, theta, sigma, rho, v0, r, T, s0, K))
print (time.time()-t1)


20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
20.596981633580796
0.08334708213806152


In [17]:
import numpy as np
from functools import partial

r = 0.01 # drift
rho = -0.4 # correlation coefficient
kappa = 3 # mean reversion coefficient
theta = 0.06 # long-term mean of the variance
sigma = 0.1 # (Vol of Vol) - Volatility of instantaneous variance
T = 1 # Terminal time
K = 100 # Stike
v0 = 0.07 # spot variance (initial variance)
S0 = 115 # spot stock price
k = np.log(K/S0)

def cf_Heston(u, t, v0, mu, kappa, theta, sigma, rho):
    """
    Heston characteristic function as proposed in the original paper of Heston (1993)
    """
    xi = kappa - sigma*rho*u*1j
    d = np.sqrt( xi**2 + sigma**2 * (u**2 + 1j*u) )
    g1 = (xi+d)/(xi-d)
    cf = np.exp( 1j*u*mu*t + (kappa*theta)/(sigma**2) * ( (xi+d)*t - 2*np.log( (1-g1 *np.exp(d*t))/(1-g1) ))\
                + (v0/sigma**2)*(xi+d) * (1-np.exp(d*t))/(1-g1*np.exp(d*t)) )
    return cf

def cf_Heston_good(u, t, v0, mu, kappa, theta, sigma, rho):
    """
    Heston characteristic function as proposed by Schoutens (2004)
    """
    xi = kappa - sigma*rho*u*1j
    d = np.sqrt( xi**2 + sigma**2 * (u**2 + 1j*u) )
    g1 = (xi+d)/(xi-d)
    g2 = 1/g1
    cf = np.exp( 1j*u*mu*t + (kappa*theta)/(sigma**2) * ( (xi-d)*t - 2*np.log( (1-g2*np.exp(-d*t))/(1-g2) ))\
                + (v0/sigma**2)*(xi-d) * (1-np.exp(-d*t))/(1-g2*np.exp(-d*t)) )
    return cf

cf_H_b = partial(cf_Heston, t=T, v0=v0, mu=r, theta=theta, sigma=sigma, kappa=kappa,rho= rho )
cf_H_b_good = partial(cf_Heston_good, t=T, v0=v0, mu=r, theta=theta, sigma=sigma, kappa=kappa, rho=rho )

def Q1(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the stock numeraire.
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( (np.exp(-u*k*1j) / (u*1j)) * cf(u-1j) / cf(-1.0000000000001j) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=2000 )[0]

def Q2(k, cf, right_lim):
    """
    P(X<k) - Probability to be in the money under the money market numeraire
    cf: characteristic function
    right_lim: right limit of integration
    """
    integrand = lambda u: np.real( np.exp(-u*k*1j) /(u*1j) * cf(u) )
    return 1/2 + 1/np.pi * quad(integrand, 1e-15, right_lim, limit=2000 )[0]

import time
limit_max = 1000 # right limit in the integration
t1 = time.time()
for i in range(1000):
    call = S0 * Q1(k, cf_H_b_good, limit_max) - K * np.exp(-r*T) * Q2(k, cf_H_b_good, limit_max)
    #put = K * np.exp(-r*T) * (1 - Q2(k, cf_H_b_good, limit_max)) - S0 * Q1(k, cf_H_b_good, limit_max)
print (time.time() - t1)
print("Heston Fourier inversion call price: ", call)
print("-----------------------------------")
print("Heston Fourier inversion put price: ", put)
print("-----------------------------------")

19.083364725112915
Heston Fourier inversion call price:  20.596981633580626
-----------------------------------
Heston Fourier inversion put price:  -57.39053690630631
-----------------------------------
